Mount your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
# MODIFY HERE TO YOUR PATH IN GOOGLE DRIVE
%cd /content/gdrive/MyDrive/CryptoML

---

Install SHAP 

In [ ]:
!pip install shap

---

Execute the functions to evaluate our models

In [ ]:
import speck as sp
import numpy as np
from tensorflow.keras.models import load_model

def create_testing_data(num_of_plaintexts, num_rounds):
  '''
  This function generates a set of testing data

  Inputs:
    - num_of_plaintexts: Number of plaintexts / num_blocks you used to train your model
    - num_rounds: Number of rounds of Speck / num_rounds you used to train your model

  Outputs:
    - X: Binary array of ciphertexts
    - Y: Relationship of ciphertexts
    - Xr: Binary array of ciphertexts having real differences
    - Yr: Relationship of ciphertexts having real differences

  '''
  assert num_of_plaintexts in [2, 4]
  assert num_rounds > 0

  if num_of_plaintexts == 2:
    X, Y = sp.make_train_data_2pt(10**6, num_rounds)
    Xr, Yr = sp.real_differences_data_2pt(10**6, num_rounds)

  else:
    X, Y = sp.make_train_data_4pt(10**6, num_rounds)
    Xr, Yr = sp.real_differences_data_4pt(10**6, num_rounds)

  return X, Y, Xr, Yr


def get_inputs(num_of_plaintexts, num_rounds, num_of_samples, model_name):
  '''
  This function is used to ensure all of the parameters modified by users have no error

  Inputs: 
    - num_of_plaintexts: Number of plaintexts
    - num_rounds: Number of rounds of Speck
    - num_of_samples: Number of samples to visualise using SHAP
    - model_name: File name of the saved model

  Outputs:
    - num_of_plaintexts: Number of plaintexts
    - num_rounds: Number of rounds of Speck
    - num_of_samples: Number of samples to visualise using SHAP
    - net: The model loaded from the file given

  '''
  assert num_of_plaintexts in [2,4]
  assert num_rounds > 0
  assert num_of_samples > 1
  assert model_name[-3:] == ".h5"

  num_of_plaintexts = num_of_plaintexts
  num_rounds = num_rounds
  net = load_model('./models/' + model_name)
  num_of_samples = num_of_samples

  return num_of_plaintexts, num_rounds, num_of_samples, net


def evaluate(net,X,Y):
  '''
  This function evaluate the model with testing data

  Inputs:
    - net: The model to evaluate
    - X: Binary array of ciphertexts
    - Y: Relationship of ciphertexts (0 or 1)

  Outputs:
  Accuracy, true positive rate, true negative rate and mean squared error of model will be printed
  Percentage of random pairs with score higher than median of real pairs will also be printed

  '''
  Z = net.predict(X,batch_size=10000).flatten()
  Zbin = (Z > 0.5)
  diff = Y - Z 
  mse = np.mean(diff*diff)
  n = len(Z) 
  n0 = np.sum(Y==0) 
  n1 = np.sum(Y==1)
  acc = np.sum(Zbin == Y) / n
  tpr = np.sum(Zbin[Y==1]) / n1
  tnr = np.sum(Zbin[Y==0] == 0) / n0
  mreal = np.median(Z[Y==1])
  high_random = np.sum(Z[Y==0] > mreal) / n0
  print("Accuracy: ", acc, "TPR: ", tpr, "TNR: ", tnr, "MSE:", mse)
  print("Percentage of random pairs with score higher than median of real pairs:", 100 * high_random)

---

Modify the parameters and run the cells to evaluate the model

In [ ]:
'''
Paramter description:
  - num_of_plaintexts: Number of plaintexts
  - num_rounds: Number of rounds of Speck
  - num_of_samples: Number of samples to visualise using SHAP
  - model_name: File name of the saved model

'''

# MODIFY HERE                                                                     ↓             ↓                 ↓                     ↓
num_of_plaintexts, num_rounds, num_of_samples, net = get_inputs(num_of_plaintexts=4, num_rounds=5, num_of_samples=10, model_name="best5depth10.h5")

In [ ]:
import shap

X, Y, Xr, Yr = create_testing_data(num_of_plaintexts, num_rounds)

In [ ]:
print('Testing neural distinguishers against blocks in the ordinary real vs random setting')
evaluate(net, X, Y)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, X[:num_of_samples])
shap_values = explainer.shap_values(X[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], X)

In [ ]:
print('Testing real differences setting now.')
evaluate(net, Xr, Yr)

shap.initjs()
explainer = shap.KernelExplainer(net.predict, Xr[:num_of_samples])
shap_values = explainer.shap_values(Xr[:num_of_samples])
shap.force_plot(explainer.expected_value[0], shap_values[0], Xr)